# Features Engeneering

In [1]:
library(dummies)
library(dplyr)
library(stringr)


load("C://bb//airbnb//data//FF_train_clean.RData")    

dim(df)

dummies-1.5.6 provided by Decision Patterns



Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union




[1] 176451     50

## Convert categorical values into dummy variables:
  
neighbourhood_cleansed - we will replace non-alphanumeric characters with underscores to prevent code errors

property_type - we will replace non-alphanumeric characters with underscores to prevent code errors

room_type - we will replace non-alphanumeric characters with underscores to prevent code errors

bed_type - we will replace non-alphanumeric characters with underscores to prevent code errors

cancellation_policy

host_is_superhost

host_identity_verified

zipcode

is_location_exact

instant_bookable

require_guest_profile_picture

require_guest_phone_verification


In [2]:
convert_categorical_to_dummies<-function(data){
    
    print('convert_categorical_to_dummies')
    
    data$neighbourhood_group_cleansed <- str_replace_all(data$neighbourhood_cleansed, "[^[:alnum:]]", "_")
  
    data$property_type <- str_replace_all(data$property_type, "[^[:alnum:]]", "_")

    data$room_type <- str_replace_all(data$room_type, "[^[:alnum:]]", "_")

    data$bed_type <- str_replace_all(data$bed_type, "[^[:alnum:]]", "_")
    
    nm<-c('neighbourhood_cleansed',
          'property_type',
          'room_type',
          'bed_type',
          'cancellation_policy',
          'host_is_superhost',
          'host_identity_verified',
          'zipcode','is_location_exact',
          'instant_bookable',
          'require_guest_profile_picture',
          'require_guest_phone_verification')
    
    data<-dummy.data.frame(data, names = nm, sep = '_')

    return(data)
}

## Convert following Orinal variables into dummy variables (due to the Missing category), other Ordinal features will be check also with numeric values

review_scores_accuracy

review_scores_cleanliness

review_scores_checkin

review_scores_communication

review_scores_location

review_scores_value

reviews_per_month

In [3]:
convert_ordinal_w_missings_to_dummies<-function(data){
    
    print('convert_ordinal_w_missings_to_dummies')
    
    nm<-c('review_scores_cleanliness', 
            'review_scores_accuracy', 
            'review_scores_checkin', 
            'review_scores_communication', 
            'review_scores_location',
            'review_scores_value',
            'reviews_per_month')

    data<-dummy.data.frame(data, names = nm, sep = '_')

    return(data)
}

### convert amenities into dummy

In [4]:
convert_amenities_to_dummy<-function(data) {
  
  print('convert_amenities_to_dummy')
  
  NUM_OF_AMENITIES<-20
  
  data$amenities <- as.character(data$amenities)
  
  data$amenities<-gsub('"', '', data$amenities)
  
  data$amenities<-gsub('\\{', '', data$amenities)
  
  data$amenities<-gsub('\\}', '', data$amenities)
  
  amenities_normilized <- gsub("(?!,)[^[:alnum:]]", "", data$amenities, perl=TRUE)
  
  amenities_normilized.split <- strsplit(amenities_normilized, ",")
  
  most.popluar.amenities<-tail(sort(table(unlist(amenities_normilized.split))), NUM_OF_AMENITIES)
  
  most.popluar.amenities.names<-names(most.popluar.amenities)
  
  m <- matrix(0, ncol = length(most.popluar.amenities.names), nrow = nrow(data))
  
  m <- data.frame(m)
  
  
  most.popluar.amenities.names.col<-paste("amenities_",most.popluar.amenities.names, sep = "")
  names(m)<-most.popluar.amenities.names.col
  
  data<-cbind(data, m)  
  
  
  for (v in 1:length(amenities_normilized)){
    for(j in most.popluar.amenities.names){
      if (length(grep(j, amenities_normilized[v])) != 0){
        data[v,paste("amenities_",j,sep = "")]<-1
      }
    }
      
    if(v %% 1000==0) {
      # Print on the screen some message
     cat(paste0("iteration: ", v, "\n"))
    }
  }
  
  return(data)
}

### Add new variables
host_about_len = ifelse(is.na(host_about), 0, nchar(host_about)) - in this case NA will be counted as 0

description_len = ifelse(is.na(description), 0, nchar(as.character(description)))

summary_len = ifelse(is.na(summary), 0, nchar(as.character(summary)))

total_amenities = count of listed amenities - since is comma delimited we will count number of separators + 1 where number of characters greater than 2 (listing brackets)

price_per_person - (avg_price/accommodates)

In [5]:
add_new_variables<-function(data){
    
    print('add_new_variables')
    
    data <- data %>% 
        mutate(host_about_len = ifelse(is.na(host_about), 0, nchar(as.character(host_about))), 
           description_len = ifelse(is.na(description), 0, nchar(as.character(description))), 
           summary_len = ifelse(is.na(summary), 0, nchar(as.character(summary))), 
           total_amenities = ifelse(nchar(as.character(amenities))>2, str_count(as.character(amenities), ',')+1, 0),
           price_per_person = avg_price / accommodates)
    
    return(data)
}

## features_engeneering function

In [6]:
features_engeneering<-function(data){
    
    data <- add_new_variables(data)
  
    data<-convert_categorical_to_dummies(data)
    
    data<-convert_ordinal_w_missings_to_dummies(data)
    
    data<-convert_amenities_to_dummy(data)
  
    return(data)
}


In [6]:
df<-add_new_variables(df)

[1] "add_new_variables"


In [7]:
df<-convert_categorical_to_dummies(df)

[1] "convert_categorical_to_dummies"


Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts ar

In [8]:
df<-convert_ordinal_w_missings_to_dummies(df)

[1] "convert_ordinal_w_missings_to_dummies"


Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"
Warning message in model.matrix.default(~x - 1, model.frame(~x - 1), contrasts = FALSE):
"non-list contrasts argument ignored"


In [10]:
df<-convert_amenities_to_dummy(df)

[1] "convert_amenities_to_dummy"
iteration: 1000
iteration: 2000
iteration: 3000
iteration: 4000
iteration: 5000
iteration: 6000
iteration: 7000
iteration: 8000
iteration: 9000
iteration: 10000
iteration: 11000
iteration: 12000
iteration: 13000
iteration: 14000
iteration: 15000
iteration: 16000
iteration: 17000
iteration: 18000
iteration: 19000
iteration: 20000
iteration: 21000
iteration: 22000
iteration: 23000
iteration: 24000
iteration: 25000
iteration: 26000
iteration: 27000
iteration: 28000
iteration: 29000
iteration: 30000
iteration: 31000
iteration: 32000
iteration: 33000
iteration: 34000
iteration: 35000
iteration: 36000
iteration: 37000
iteration: 38000
iteration: 39000
iteration: 40000
iteration: 41000
iteration: 42000
iteration: 43000
iteration: 44000
iteration: 45000
iteration: 46000
iteration: 47000
iteration: 48000
iteration: 49000
iteration: 50000
iteration: 51000
iteration: 52000
iteration: 53000
iteration: 54000
iteration: 55000
iteration: 56000
iteration: 57000
iterati

In [13]:
dim(df)

filename_clean_enriched<-"./data/FF_train_clean_enriched.csv"

con<-file(filename_clean_enriched,encoding="UTF-8")

write.csv(df,file=con, row.names = FALSE)

save(df, file=gsub(".csv", ".RData", filename_clean_enriched))

[1] 176451    240

#remove string variables
string_features<-c('summary','space', 'description', 'neighborhood_overview','notes','transit','host_about','amenities')
big.factors<-c('neighbourhood_cleansed','zipcode', 'property_type')
no.big.factors<-setdiff(names(df), c(string_features, big.factors, "occupied.cat")) 
df<-df[no.big.factors]

# convert all the factors into numberic (in order to run clustering)
indx <- sapply(dft, is.factor)
df[indx] <- lapply(dft[indx], function(x) as.numeric(x))

You can use clara algorithm, it is implemented in R (package cluster). It works by clustering a random sample from the dataset and then assign all elements in the dataset to these clusters. By the way I've never used it for matrix with 80.000 rows, but it works quite well for 50.000.